In [17]:
// https://www12.senado.leg.br/transparencia/dados-abertos-transparencia/dados-abertos-ceaps

#r "nuget: FSharp.Data,6.0.1-beta001"
#r "nuget: Plotly.NET.Interactive"
#r "nuget: Plotly.NET.ImageExport"

open FSharp.Data
open System
open Plotly.NET
open Plotly.NET.ImageExport

[<Literal>]
let ResolutionFolder = __SOURCE_DIRECTORY__

type Despesas = CsvProvider<"./despesa_ceaps_2022.csv", ";", ResolutionFolder = ResolutionFolder>

let despesas = Despesas.Load(ResolutionFolder + "/despesa_ceaps_2022.csv")

let Senador = "MARCOS DO VAL"

let parseDecimal (value:String) = Decimal.Parse(value.Replace(',','.'))

let parseDate (value:String) = DateTime.ParseExact(value, "dd/MM/yyyy", null)

let data = 
    despesas.Rows
    |> Seq.filter(fun d -> d.SENADOR = Senador && d.ANO = 2022)

Installed Packages FSharp.Data, 6.0.1-beta001 Plotly.NET.ImageExport, 3.0.1 Plotly.NET.Interactive, 3.0.2

In [ ]:
data
|> Seq.groupBy(fun it -> parseDate it.DATA)

In [20]:
data
|> Seq.groupBy(fun it -> it.TIPO_DESPESA)
|> Seq.map( fun (supplier, expenses) ->
    let dates, values =
        expenses
        |> Seq.map(fun it -> ( parseDate it.DATA), (it.VALOR_REEMBOLSADO |> parseDecimal))
        |> Seq.sortBy fst
        |> Seq.toArray
        |> Array.unzip
    Chart.Point(dates, values, Name = supplier)
)
|> Chart.combine
|> Chart.withSize(1400, 1000)
|> Chart.withLegendStyle(Orientation = StyleParam.Orientation.Horizontal)
|> Chart.withTitle("Despesas com cota parlamentar " + Senador)


<!-- Plotly chart will be drawn inside this DIV -->